In [ ]:
using Revise

In [ ]:
push!(LOAD_PATH, ".");

In [ ]:
using Images
using VideoIO: openvideo
using ImageView: imshow
using StaticArrays: SVector, SMatrix
using CoordinateTransformations
using Base.Test
using BenchmarkTools
using Interact
using Interpolations
using Unitful

usable motion from 3s to 20s

In [ ]:
import Cornercam
reload("Cornercam")

In [ ]:
video = VideoIO.openvideo("DSCF7139.MOV")
source = Cornercam.TransformedVideo(video) do frame
    restrict(restrict(frame[200:end, 330:955]))
end;

In [ ]:
background = mean(Cornercam.frames_between(source, (10u"s", 12u"s")))

In [ ]:
original_corners = [[10, 22], 
    [12, 148], 
    [131, 155], 
    [123, 5]]

desired_corners = [[0, 0], 
    [0, 1],
    [1, 1],
    [1, 0]]

In [ ]:
H = Cornercam.rectify(desired_corners, original_corners)
@test all((H.(desired_corners) .≈ original_corners))

In [ ]:
samples = Cornercam.polar_samples(
    linspace(0, π/2, 200),
    linspace(0.01, 1, 50)
);

θs = linspace(0, π/2, 200)
σ = 0.085
params = Cornercam.Params(θs, samples, σ)
cam = Cornercam.CornerCamera(source, H, background, params);

In [ ]:
Cornercam.show_samples(cam)

In [ ]:
A = Cornercam.visibility_gain(samples, θs);

In [ ]:
λ = sqrt(2.7)
gain = Cornercam.cornercam_gain(A, σ, λ);

In [ ]:
background_samples = Cornercam.sample(background, samples, H)

In [ ]:
seek(source, 4.0)
buf = read(source)

trace = hcat(map(1:160) do i
        for i in 1:6
            read!(source, buf)
        end
        pixels = Cornercam.sample(buf, samples, H)
        pixels .-= background_samples
        y = reshape(pixels, :)
        Lvx =  gain *  y
        x = Lvx[2:end]
        x
        end...)

In [ ]:
Cornercam.imnormal(trace)